In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import time
import csv


In [2]:
folder_path = r'D:\Yash-python\python\Tasks\NGSpurs\Images'

file_list = os.listdir(folder_path)

In [3]:
kernel_size=5
std_dev=1.5

In [4]:
# Define the headings
headings = [
    "Distance in Pixels",
    "Broken",
    "Code_Duration"
]


In [5]:
with open("output.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headings)
    start = time.time()
    # Loop through each file in the folder
    for file in file_list:
        start_img = time.time()
        file_path = os.path.join(folder_path, file)
        
        # Step 1: Read and process the image
        img = cv2.imread(file_path)
        img = cv2.resize(img, (500, 500))
        
        # Step 2: Create Gaussian filter
        kernel = cv2.getGaussianKernel(kernel_size, std_dev)
        kernel = kernel * kernel.T
        fil_img = cv2.filter2D(img, -1, kernel)
        
        # Step 3: Convert to grayscale and detect edges
        gray = cv2.cvtColor(fil_img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, threshold1=100, threshold2=200)
        
        # Step 4: Find contours
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        X, Y = [], []
        ck = False
        min_x, max_x = 500, 0

        # Step 5: Process contours to calculate min_x and max_x
        for contour in contours:
            for point in contour:
                x, y = point[0]
                X.append(x)
                Y.append(y)
                min_x = min(min_x, x)
                max_x = max(max_x, x)

        # Step 6: Draw contours on the image
        edge_detect = cv2.drawContours(fil_img.copy(), contours, -1, (0, 255, 0), 2)
        
        # Step 8: Check if contour spans a wide range
        if max_x - min_x > 3:
            ck = True

        end_img = time.time()
        duration = (end_img - start_img) * 1000  # Convert to milliseconds
        cv2.imshow("Image",edge_detect)
        # Step 10: Write to CSV
        writer.writerow([max_x-min_x,ck, f"{duration:.3f} ms"])
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    end = time.time()
    print((end - start) * 1000, "ms")
print("CSV file 'output.csv' created successfully.")

147187.55316734314 ms
CSV file 'output.csv' created successfully.
